# Пробуем простую полносвязную сеть

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
plt.style.use('seaborn')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path
from skimage import transform
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
from read_data import *
from read_data import good_cols
%matplotlib inline

C:\Users\kitro\anaconda3\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
all_df = get_df()
activities = sep_by_len(all_df, 20000)
cut_df = cut_act(activities, 20000)
n_df = normalize_df(cut_df)
n_df.sample()

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:33<00:00,  3.73s/it]


,activityID,heart rate,temperature hand,3Da_x scale_16 hand,3Da_y scale_16 hand,3Da_z scale_16 hand,3Dg_x hand,3Dg_y hand,3Dg_z hand,3Dm_x hand,...,temperature ankle,3Da_x scale_16 ankle,3Da_y scale_16 ankle,3Da_z scale_16 ankle,3Dg_x ankle,3Dg_y ankle,3Dg_z ankle,3Dm_x ankle,3Dm_y ankle,3Dm_z ankle
37,7,"[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.062...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.5807687784651435, 0.5875660042887362, 0.592...","[0.28103391128129085, 0.28204211470535184, 0.2...","[0.42413333449688667, 0.4326688716004241, 0.43...","[0.442983216579101, 0.41466287215411557, 0.386...","[0.8654766505157396, 0.8727275815650121, 0.861...","[0.6100960800600779, 0.624218126690715, 0.6420...","[0.6133390566248151, 0.597803055648489, 0.5847...",...,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.2943924997534028, 0.29856059712339117, 0.29...","[0.6460575419653657, 0.646604124258784, 0.6465...","[0.4937778510383286, 0.4948768058648176, 0.497...","[0.4896060059425519, 0.4909526148010758, 0.488...","[0.5892198590882386, 0.5903698419058085, 0.585...","[0.4654053591766975, 0.4646651159381138, 0.462...","[0.29610254837395705, 0.3042320221391264, 0.30...","[0.776323860977796, 0.7734719472547551, 0.7774...","[0.6292756420449089, 0.6360089861885369, 0.627..."


Получаем вектор признаков

In [5]:
X = n_df.iloc[:, 1:].values

Y = n_df.iloc[:, 0] # целевая переменная
X.shape, Y.shape

((64, 31), (64,))

In [ ]:
def get_flatten(data):
    flat = []
    for el in data:
        flat += list(el.flatten())

In [ ]:
X_flat = get_flatten(X_data, resize_len=target_len)
X_flat.shape